# Assignment 10: Neural Networks

Only use the already imported library and the Python standard library. For the evaluation you may also use scikit-learn (`sklearn`) and `matplotlib`. Make sure that the dataset `Churn_Modelling.csv` is in the same directory as the notebook.

List your team members (name and immatriculation number) and indicate whether you are a B.Sc. Data Science or other group in the following cell:

==> *Write*
* Saurabh J
* 3574931
* MSc Info-Tech

* Erick Villanueva
* 3731779
* Computational Linguistics MSc.

* Song Cheng
* 3627396
* Autonome Systeme MSc

*of all assignment group participants here. (double klick here to edit)*

In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt

## Task 3: Feedforward Neural Network: Programming (40 Points)

In this task, you will implement a feedforward neural network for binary classification. The hyperparameters of the model are:
- `input_size`: The dimension of the input vector.
- `output_size`: The dimension of the output vector, which is 1 in this case.
- `hidden_layers`: A list of integers where each integer represents the number of neurons in that hidden layer.
- `learning_rate`: The learning rate for gradient descent.
- `epochs`: The number of epochs/iterations performed during training.

B.Sc. Data Science only have to implement for a single hidden layer. All other students have to implement the network for any length of hidden_layers.

The activation function for every layer is sigmoid function.

You have to implement the `FeedforwardNeuralNetworkClassifier`.

The `fit` method trains the network.
Use backpropagation with gradient descent.
Use the whole training data set for each training epoch.
Use the mean squared error as loss function.

The `predict` method computes the forward-pass of the network.

Evaluate your classifier on the test data with the mean squared error. Try out different hyper-parameters and compare the results. 

In [17]:
class FeedforwardNeuralNetworkClassifier:
    def __init__(self, input_size, hidden_layers, output_size=1, learning_rate=0.01, epochs=1000):
        self.lr = learning_rate
        self.epochs = epochs
        self.input_size = input_size
        self.hidden_layers = hidden_layers
        self.output_size = output_size
        
        # Initialize weights and biases
        self.weights = []
        self.biases = []

        layer_sizes = [input_size] + hidden_layers + [output_size]
        for i in range(len(layer_sizes) - 1):
            self.weights.append(np.random.rand(layer_sizes[i], layer_sizes[i+1]))
            self.biases.append(np.zeros((1, layer_sizes[i+1])))

    def sigmoid(self, x):
        z = np.exp(-x)
        return 1 / (1 + z)
        
    def sigmoid_derivative(self, x):
        sig = self.sigmoid(x)
        return sig * (1.0 - sig)
    
    


    def train(self, X_train, y_train):
        for epoch in range(self.epochs):
            # Forward pass
            activations, zs = self.forward(X_train)
            y_pred = activations[-1]

            # Compute loss
            current_loss = np.mean(y_pred - y_train)

            # Backward pass to compute gradients
            nabla_w, nabla_b = self.backward(X_train, y_train, activations, zs)

            # Update weights and biases
            for i in range(len(self.weights)):
                self.weights[i] -= self.lr * nabla_w[i]
                self.biases[i] -= self.lr * nabla_b[i]

            # Print epoch and loss
            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Loss: {current_loss}")

    def forward(self, X):
        activations = [X]
        zs = []

        for i in range(len(self.weights)):
            z = activations[-1] @ self.weights[i] + self.biases[i]
            zs.append(z)
            activations.append(self.sigmoid(z))
        
        return activations, zs

In [14]:
# Load dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [15]:
# Encode categorical data
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

# One Hot Encoding the "Geography" column
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Feature Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [16]:
# Train the network
hidden_layers = [32, 16, 8]  # Example hidden layers with 10 neurons each
learning_rate = 0.01
epochs = 1000

nn = FeedforwardNeuralNetworkClassifier(input_size=X_train.shape[1], hidden_layers=hidden_layers, learning_rate=learning_rate, epochs=epochs)
nn.train(X_train, y_train)



ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1 is different from 8000)

In [ ]:
# Evaluate the network
y_pred = nn.predict(X_test)
accuracy = np.mean(y_pred == y_test.reshape(-1, 1))
print(f'Accuracy: {accuracy * 100:.2f}%')

# Plot the training error over epochs
plt.plot(nn.training_errors)
plt.xlabel('Epochs')
plt.ylabel('Training Error')
plt.title('Training Error over Epochs')
plt.show()